In [1]:
print("hello")

hello


In [4]:
# Import necessary libraries
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader

In [6]:
!pip install pycocotools

     |████████████████████████████████| 169 kB 943 kB/s eta 0:00:01


In [7]:
import os
from pycocotools.coco import COCO
import requests
import zipfile

# Define the directory where you want to store the dataset
data_dir = 'coco_data'

# Create the data directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# URLs for the COCO dataset
train_image_url = "http://images.cocodataset.org/zips/train2017.zip"
val_image_url = "http://images.cocodataset.org/zips/val2017.zip"
annotations_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

# Download and extract the train and validation images
for url in [train_image_url, val_image_url, annotations_url]:
    response = requests.get(url, stream=True)
    zip_file_path = os.path.join(data_dir, os.path.basename(url))
    with open(zip_file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(zip_file_path)

# Initialize the COCO API for annotations
annotations_dir = os.path.join(data_dir, "annotations")
json_path = os.path.join(annotations_dir, "instances_train2017.json")
coco = COCO(json_path)

# You can now use the COCO API to work with the downloaded dataset



KeyboardInterrupt: 

In [ ]:
# Import necessary libraries
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader

# Define data transformations
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

# Load the COCO dataset (training set)
coco_train_dataset = CocoDetection(root="path/to/coco/train2017", annFile="path/to/coco/annotations/instances_train2017.json", transform=transform)

# Create a DataLoader for training
train_dataloader = DataLoader(coco_train_dataset, batch_size=2, shuffle=True, num_workers=4)

# Define the YOLO model (You can use YOLOv3 or other YOLO variants)
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Set training parameters and optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train the model
model.train()
for epoch in range(10):
    for images, targets in train_dataloader:
        images = list(image for image in images)
        targets = [{k: v for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    lr_scheduler.step()

# Save the trained model
torch.save(model.state_dict(), "yolo_coco_model.pth")
